In [1]:
from __future__ import division, print_function, absolute_import

# Import tflearn and some helpers
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import pickle


C:\Users\smith\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
dataset_path="C:\\Users\\smith\\Documents\\Becky\\HawkWatch\\BirdRecognition\\"
# Load the data set
X, Y, X_test, Y_test = pickle.load(open(dataset_path+"full_dataset.pkl", "rb"), encoding='bytes')


In [4]:
# Shuffle the data
X, Y = shuffle(X, Y)



In [9]:
#print(X[1])
#print(Y[1])

In [11]:
# Make sure the data is normalized
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()


In [12]:
# Create extra synthetic training data by flipping, rotating and blurring the
# images on our data set.
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)


In [13]:
print(img_aug)

In [14]:
# Define our network architecture:

# Input is a 32x32 image with 3 color channels (red, green and blue)
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)


In [15]:
print(network)

Tensor("InputData/X:0", shape=(?, 32, 32, 3), dtype=float32)


In [16]:
# Step 1: Convolution
network = conv_2d(network, 32, 3, activation='relu')

# Step 2: Max pooling
network = max_pool_2d(network, 2)


In [17]:
# Step 3: Convolution again
network = conv_2d(network, 64, 3, activation='relu')

# Step 4: Convolution yet again
network = conv_2d(network, 64, 3, activation='relu')



In [18]:
# Step 5: Max pooling again
network = max_pool_2d(network, 2)

# Step 6: Fully-connected 512 node neural network
network = fully_connected(network, 512, activation='relu')


In [19]:
print(network)

Tensor("FullyConnected/Relu:0", shape=(?, 512), dtype=float32)


In [20]:
# Step 7: Dropout - throw away some data randomly during training to prevent over-fitting
network = dropout(network, 0.5)

# Step 8: Fully-connected neural network with two outputs (0=isn't a bird, 1=is a bird) to make the final prediction
network = fully_connected(network, 2, activation='softmax')


In [21]:
print(network)

Tensor("FullyConnected_1/Softmax:0", shape=(?, 2), dtype=float32)


In [22]:
# Tell tflearn how we want to train the network
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)




In [23]:
# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='bird-classifier.tfl.ckpt')



In [24]:
print(network)

Tensor("FullyConnected_1/Softmax:0", shape=(?, 2), dtype=float32)


In [25]:
# Train it! We'll do 100 training passes and monitor it as it goes.
model.fit(X, Y, n_epoch=100, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96,
          snapshot_epoch=True,
          run_id='bird-classifier')

Training Step: 59199  | total loss: 0.17539 | time: 247.311s
| Adam | epoch: 100 | loss: 0.17539 - acc: 0.9307 -- iter: 56736/56780
Training Step: 59200  | total loss: 0.17509 | time: 269.574s
| Adam | epoch: 100 | loss: 0.17509 - acc: 0.9313 | val_loss: 0.19254 - val_acc: 0.9451 -- iter: 56780/56780
--
INFO:tensorflow:C:\Users\smith\Documents\Becky\Hawk Watch\BirdRecognition\bird-classifier.tfl.ckpt-59200 is not in all_model_checkpoint_paths. Manually adding it.


In [26]:
# Save model when training is complete to a file
output_path="C:\\Users\\smith\\Documents\\Becky\\HawkWatch\\BirdRecognition\\"
model.save(output_path+"bird-classifier.tfl")
print("Network trained and saved as bird-classifier.tfl!")


INFO:tensorflow:C:\Users\smith\Documents\Becky\Hawk Watch\BirdRecognition\bird-classifier.tfl is not in all_model_checkpoint_paths. Manually adding it.
Network trained and saved as bird-classifier.tfl!


In [27]:
# Save model when training is complete to a file
output_path="C:\\Users\\smith\\Documents\\Becky\\HawkWatch\\BirdRecognition\\"
model.save(output_path+"another-bird-classifier.tfl")
print("Network trained and saved as another-bird-classifier.tfl!")

INFO:tensorflow:C:\Users\smith\Documents\Becky\Hawk Watch\BirdRecognition\another-bird-classifier.tfl is not in all_model_checkpoint_paths. Manually adding it.
Network trained and saved as another-bird-classifier.tfl!
